In [35]:
import os
import re
import csv
import random
from glob import glob

import numpy as np
import pandas as pd

from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import torchvision.transforms as v2
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.cli import tqdm

import warnings
warnings.filterwarnings('ignore')


In [36]:
def seed_everything(seed=42):
    random.seed(seed)  # Python 내장 random 모듈
    os.environ['PYTHONHASHSEED'] = str(seed)  # 환경변수 설정
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # PyTorch CPU 시드 고정
    torch.cuda.manual_seed(seed)  # PyTorch GPU 시드 고정
    torch.cuda.manual_seed_all(seed)  # 멀티 GPU 환경에서도 시드 고정
    torch.backends.cudnn.deterministic = True  # CuDNN 관련 설정
    torch.backends.cudnn.benchmark = False  # 동일한 입력 크기의 데이터가 반복될 경우 속도 향상을 위한 벤치마크 모드 비활성화

# 사용 예시
seed_everything(seed=42)

이미지 경로 포함

In [15]:
valid_df = pd.read_csv('datasets/val_label.csv')
valid_df['data_path'] = valid_df.apply(lambda row: f"temp/temp{row['subject_id']}_{row['date']}.png", axis=1)

In [16]:
valid_df.head()

,subject_id,date,Q1,Q2,Q3,S1,S2,S3,S4,data_path
0,1,2023-08-20,1,1,1,0,0,0,0,temp/temp1_2023-08-20.png
1,1,2023-08-21,1,1,1,0,0,1,0,temp/temp1_2023-08-21.png
2,1,2023-08-22,0,1,1,0,1,1,0,temp/temp1_2023-08-22.png
3,1,2023-08-23,0,1,1,0,0,1,0,temp/temp1_2023-08-23.png
4,1,2023-08-24,1,1,1,0,0,1,0,temp/temp1_2023-08-24.png


In [17]:
class CustomDataset(Dataset):
    def __init__(self, df, transforms):
        self.path = df['data_path'].values
        self.class_ = df[['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3', 'S4']].values
        self.transform = transforms

    def __getitem__(self, idx):
        try:
            img = np.array(Image.open(self.path[idx]).convert('RGB'))
        except FileNotFoundError:
            return self.__getitem__((idx + 1) % len(self))
        
        img = self.transform(image=img)['image']
        img = torch.tensor(img, dtype=torch.float)  # Explicitly specify the type as float
        y = self.class_[idx]
        
        return img, y
    
    def __len__(self):
        return len(self.path)

### 데이터 mean, std 계산

In [19]:
import numpy as np
import albumentations as A
from albumentations.core.transforms_interface import ImageOnlyTransform
from albumentations.pytorch import ToTensorV2

# 기존의 train_transforms에 새로운 증강 기법 추가
train_transforms = A.Compose([
    A.Resize(always_apply = True, p=1.0, height=224, width=224),
    A.Normalize(p=1.0),
    ToTensorV2()
])


test_transforms = A.Compose([
    A.Resize(always_apply = True, p=1.0, height=224, width=224),
    A.Normalize(p=1.0),
    ToTensorV2()
])

In [20]:
epochs = 200

In [31]:
from collections import deque

def run_model(model, loader, loss_fn=None, optimizer=None, is_training=False, epoch=None):
    targets = []
    preds = []
    smooth_loss_queue = deque(maxlen=50)  # 최근 50개의 손실을 저장할 큐

    if is_training:
        model.train()
        mode = 'Train'
    else:
        model.eval()
        mode = 'Valid/Test'

    running_loss = 0.0
    bar = tqdm(loader, ascii=True, leave=False)
    for cnt, (data, target) in enumerate(bar):
        data = data.to('cuda')
        target = target.to('cuda')
        if is_training:
            optimizer.zero_grad()

        outputs = torch.sigmoid(model(data))
        total_loss = loss_fn(outputs, target.float())
        running_loss += total_loss.item()
        smooth_loss_queue.append(total_loss.item())  # 현재 손실을 큐에 추가
        smooth_loss = sum(smooth_loss_queue) / len(smooth_loss_queue)  # 큐에 있는 손실의 평균 계산

        predicted = (outputs > 0.5).float()
        preds.extend(predicted.detach().cpu().tolist())
        targets.extend(target.detach().cpu().tolist())

        if is_training:
            total_loss.backward()
            optimizer.step()

        # 에폭 정보와 함께 배치별 손실 평균 및 smooth loss 출력
        bar.set_description(f'Epoch {epoch} {mode} - Loss: {total_loss:.4f}, Smooth Loss: {smooth_loss:.4f}')

    f1_score_ = f1_score(np.array(targets), np.array(preds), average='macro')
    acc_score = accuracy_score(np.array(targets).reshape(-1), np.array(preds).reshape(-1))

    return running_loss / len(loader), acc_score, f1_score_


In [32]:
full_df = pd.read_csv('datasets/val_label.csv')
full_df['data_path'] = full_df.apply(lambda row: f"datasets/image_datasets/valid_images/user{row['subject_id']}_{row['date']}_org.png", axis=1)

In [33]:
import logging
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn import BCELoss
from torchvision import transforms
from sklearn.model_selection import KFold
import timm

full_dataset = CustomDataset(full_df, train_transforms)  # full_df는 전체 데이터프레임을 나타냅니다.

kf = KFold(n_splits=5, shuffle=True, random_state=1020)

fold_perf = {}

for fold, (train_idx, valid_idx) in enumerate(kf.split(full_dataset)):
    logging.info(f"Starting Fold {fold+1}")
    
    # 데이터셋 분할
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_idx)
    
    # 데이터 로더 설정
    train_loader = DataLoader(full_dataset, batch_size=16, sampler=train_subsampler, num_workers=8)
    valid_loader = DataLoader(full_dataset, batch_size=16, sampler=valid_subsampler, num_workers=8)
    
    # 모델 초기화 및 이동
    model = timm.create_model('seresnext101_32x4d', pretrained=True, num_classes=7)
    model = model.to('cuda')
    
    # 손실 함수, 최적화, 스케줄러 설정
    criterion = BCELoss()
    optimizer = optim.AdamW(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=50, T_mult=1, eta_min=0.00007)
    
    best_f1 = -float('inf')
    best_loss = 0

    # 학습 및 검증
    for epoch in range(epochs):
        train_loss, train_acc, train_f1 = run_model(model, train_loader, criterion, optimizer, is_training=True, epoch=epoch)
        valid_loss, valid_acc, valid_f1 = run_model(model, valid_loader, criterion, optimizer, is_training=False, epoch=epoch)
        
        print(f'Epoch {epoch+1}: Train Loss: {train_loss:.6f}, Train Acc: {train_acc:.6f}, Train F1: {train_f1:.6f}, Valid Loss: {valid_loss:.6f}, Valid Acc: {valid_acc:.6f}, Valid F1: {valid_f1:.6f}')
        
        if valid_f1 > best_f1:
            best_f1 = valid_f1
            best_loss = valid_loss
            model_path = f'result/fold_{fold}_f1-{best_f1:.3f}_loss-{best_loss:.3f}_S1check.pt'
            torch.save(model.state_dict(), model_path)
            logging.info(f'New best F1-score {valid_f1:.4f} achieved at epoch {epoch}, model saved at {model_path}')
        
        scheduler.step()
    
    # 각 fold의 성능 기록
    fold_perf[fold] = {
        'train_loss': train_loss,
        'train_acc': train_acc,
        'train_f1': train_f1,
        'valid_loss': valid_loss,
        'valid_acc': valid_acc,
        'valid_f1': valid_f1
    }

Epoch 1: Train Loss: 0.664333, Train Acc: 0.598639, Train F1: 0.535600, Valid Loss: 0.639699, Valid Acc: 0.612245, Valid F1: 0.490150


Epoch 2: Train Loss: 0.494915, Train Acc: 0.860544, Train F1: 0.793136, Valid Loss: 0.595950, Valid Acc: 0.700680, Valid F1: 0.484976


Epoch 3: Train Loss: 0.366654, Train Acc: 0.935374, Train F1: 0.892581, Valid Loss: 0.568097, Valid Acc: 0.714286, Valid F1: 0.479742


Epoch 4: Train Loss: 0.268805, Train Acc: 0.964286, Train F1: 0.944017, Valid Loss: 0.483221, Valid Acc: 0.727891, Valid F1: 0.484378


Epoch 5: Train Loss: 0.205410, Train Acc: 0.982993, Train F1: 0.976263, Valid Loss: 0.523295, Valid Acc: 0.761905, Valid F1: 0.542935


...

Epoch 196: Train Loss: 0.001979, Train Acc: 1.000000, Train F1: 1.000000, Valid Loss: 0.645052, Valid Acc: 0.632653, Valid F1: 0.574425


Epoch 197: Train Loss: 0.002498, Train Acc: 1.000000, Train F1: 1.000000, Valid Loss: 0.641017, Valid Acc: 0.653061, Valid F1: 0.545831


Epoch 198: Train Loss: 0.001628, Train Acc: 1.000000, Train F1: 1.000000, Valid Loss: 0.640113, Valid Acc: 0.687075, Valid F1: 0.620197


Epoch 199: Train Loss: 0.004471, Train Acc: 1.000000, Train F1: 1.000000, Valid Loss: 0.625859, Valid Acc: 0.673469, Valid F1: 0.562353


Epoch 200: Train Loss: 0.002683, Train Acc: 1.000000, Train F1: 1.000000, Valid Loss: 0.612798, Valid Acc: 0.673469, Valid F1: 0.620457


In [34]:
import timm

resnet_models = timm.list_models()

for model_name in resnet_models:
    print(model_name)

bat_resnext26ts
beit_base_patch16_224
beit_base_patch16_384
beit_large_patch16_224
beit_large_patch16_384
beit_large_patch16_512
beitv2_base_patch16_224
beitv2_large_patch16_224
botnet26t_256
botnet50ts_256
caformer_b36
caformer_m36
caformer_s18
caformer_s36
cait_m36_384
cait_m48_448
cait_s24_224
cait_s24_384
cait_s36_384
cait_xs24_384
cait_xxs24_224
cait_xxs24_384
cait_xxs36_224
cait_xxs36_384
coat_lite_medium
coat_lite_medium_384
coat_lite_mini
coat_lite_small
coat_lite_tiny
coat_mini
coat_small
coat_tiny
coatnet_0_224
coatnet_0_rw_224
coatnet_1_224
coatnet_1_rw_224
coatnet_2_224
coatnet_2_rw_224
coatnet_3_224
coatnet_3_rw_224
coatnet_4_224
coatnet_5_224
coatnet_bn_0_rw_224
coatnet_nano_cc_224
coatnet_nano_rw_224
coatnet_pico_rw_224
coatnet_rmlp_0_rw_224
coatnet_rmlp_1_rw2_224
coatnet_rmlp_1_rw_224
coatnet_rmlp_2_rw_224
coatnet_rmlp_2_rw_384
coatnet_rmlp_3_rw_224
coatnet_rmlp_nano_rw_224
coatnext_nano_rw_224
convformer_b36
convformer_m36
convformer_s18
convformer_s36
convit_base
conv